In [1]:
import pandas as pd
import awswrangler as awr
import openpyxl
import os
import shutil
import unicodedata
import win32com.client as win32

#-----------------EXTRACT
path_to_query = r"C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\base_seguro_de_vida.sql"

with open (path_to_query,'r') as reading:
    query_readed = reading.read()

df = awr.athena.read_sql_query(query_readed,'silver')

In [ ]:
#-----------------TRANSFORM
#adding empty e-mails with the e-mail common pattern
row = df['e_mail'].isnull() & df['unidade'].str.contains('UNIDADE ')
df.loc[row, 'e_mail'] = 'unidade.' + df.loc[row, 'unidade'].str.replace('UNIDADE ', '').str.lower() + '@transdesk.com.br'


,cooperativa,unidade,cliente,conjunto,coverage,beneficio,placa,motorista,e_mail
0,Stcoop,UNIDADE VILHENA,DIONE MAFRA DE SOUZA,11084,<NA>,<NA>,<NA>,<NA>,unidade.vilhena@transdesk.com.br
1,Stcoop,UNIDADE VILHENA,DIONE MAFRA DE SOUZA,7443,<NA>,<NA>,<NA>,<NA>,unidade.vilhena@transdesk.com.br
2,Stcoop,UNIDADE VILHENA,DIONE MAFRA DE SOUZA,6644,<NA>,<NA>,<NA>,<NA>,unidade.vilhena@transdesk.com.br
3,Stcoop,UNIDADE VILHENA,DIONE MAFRA DE SOUZA,6402,<NA>,<NA>,<NA>,<NA>,unidade.vilhena@transdesk.com.br
4,Stcoop,UNIDADE VILHENA,DIONE MAFRA DE SOUZA,6396,<NA>,<NA>,<NA>,<NA>,unidade.vilhena@transdesk.com.br


In [3]:
#removing empty spaces
df['e_mail'] = df['e_mail'].str.replace(' ','')

#removing diacritics with unicodedata
def remove_diacritics (texto):
    if isinstance(texto,str):
        return ''.join(c for c in unicodedata.normalize('NFKD',texto) if not unicodedata.combining(c))
    return texto

df['e_mail'] = df['e_mail'].apply(remove_diacritics)

df = df.reset_index(drop=True)


In [24]:
df_unidemail = df.groupby(['unidade', 'e_mail']).agg(
    placas=('placa', 'count'),
    motoristas=('motorista', 'count')
).reset_index()


df_unidemail_filtered = df_unidemail[df_unidemail['placas']>df_unidemail['motoristas']]


,unidade,e_mail,placas,motoristas
1,TS CONSULTORIA EM TRANSPORTES LTDA,contasapagar.unus@gmail.com,939,218
2,UNIDADE ARACAJU,jairo.torres@transdesk.com.br,7129,723
3,UNIDADE ARACAJU,maria.olga@transdesk.com.br,7129,723
4,UNIDADE ARACAJU,unidade.aracaju@transdesk.com.br,597,112
5,UNIDADE CAMPO GRANDE,unidade.campogrande@transdesk.com.br,1772,88
6,UNIDADE CARAZINHO,rafael.sartori@transdesk.com.br,65,8
7,UNIDADE CARAZINHO,unidade.carazinho@transdesk.com.br,1516,150
8,UNIDADE CASCAVEL,moises.junior@transdesk.com.br,28452,1755
9,UNIDADE CASCAVEL,unidade.cascavel@transdesk.com.br,10095,964
10,UNIDADE CHAPECÓ,angela.padilha@transdesk.com.br,3164,247


In [23]:
#-----------------LOAD
#Function to send e-mails

def enviar_email(p_email, p_unidade, p_placas, p_motoristas):    
    outlook = win32.Dispatch('outlook.application')  #integrating python with outlook 

    #creating e-mail variable to use
    email = outlook.CreateItem(0)

    #configuring the e-mail
    email.To = p_email
    email.Subject = "[CADASTRAMENTO DE MOTORISTAS] - Análise de Dados"
    email.HTMLBody = f"""
    <p>Prezado(a),<p/>

    <p>A unidade {p_unidade}, correspondente ao e-mail {p_email}, possui pendências de cadastramento de motoristas por placa:

    <p>Número de placas cadastradas: {p_placas}
    Número de motoristas cadastrados: {p_motoristas}<p/>

    <p>Favor, cadastrar as placas remanescentes para cômputo na base de dados da empresa, obrigado!<p/>

    <p>Atenciosamente,<p/>

    <p>Equipe Análise de Dados - Grupo Unus<p/>

    <p>(<i>Este é um e-mail automático, por favor não responda<i/>)<p/>
 
    """

    email.Send()

    print("Email enviado")

In [21]:

df_numeros = df_unidemail_filtered.groupby(['unidade']).agg(
    qtd_placas=('placas', 'sum'),
    qtd_motoristas=('motoristas', 'sum')
).reset_index()

for idx, row in df_unidemail_filtered.iterrows():
    iEmail = row['e_mail']
    iUnidade = row['unidade']

    unidade_info = df_numeros.loc[df_numeros['unidade'] == iUnidade].iloc[0]

    iQtdPlacas = unidade_info['qtd_placas']
    iQtdMotoristas = unidade_info['qtd_motoristas']

    if __name__=="__main__":
        enviar_email(iEmail, iUnidade, iQtdPlacas, iQtdMotoristas)

    
    print(f"e_mail:{iEmail}/unidade:{iUnidade}/placas:{iQtdPlacas}/motoristas:{iQtdMotoristas}")


        

NameError: name 'iContato' is not defined

In [ ]:


path_to_save = r'C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista'
file_name = 'Cobertura de Seguros Motoristas.xlsx'
final_path = os.path.join(path_to_save, file_name)
os.remove(final_path)
df.to_excel(final_path, engine='openpyxl', index = False)